In [1]:
# 쿠팡크롤링 통합 버전
import time
import logging
import pandas as pd
import os
import subprocess
import sys
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, JavascriptException

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def setup_driver():
    options = Options()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    return webdriver.Chrome(options=options)

def safe_find_text(element, by, value, default="-"):
    try:
        found_element = element.find_element(by, value)
        return found_element.text if found_element else default
    except NoSuchElementException:
        return default

def scrape_review(driver, review_element, index):
    review_data = {'순번': index + 1}
    
    logger.info("리뷰를 스크래핑하고 있습니다...")

    # 리뷰어 이름
    review_data['리뷰어_이름'] = safe_find_text(review_element, By.CSS_SELECTOR, ".sdp-review__article__list__info__user__name")

    # 리뷰어 배지 (예: Vine 프로그램)
    reviewer_badge = safe_find_text(review_element, By.CSS_SELECTOR, ".sdp-review__article__list__info__adventure-badge")
    review_data['체험단_리뷰어'] = 'VINE' in reviewer_badge

    # 별점
    star_element = review_element.find_element(By.CSS_SELECTOR, ".sdp-review__article__list__info__product-info__star-orange")
    style = star_element.get_attribute("style") if star_element else ""
    width = style.split("width:")[1].split("%")[0] if "width:" in style else "0"
    review_data['평점'] = float(width) / 20 if width else "-"

    # 리뷰 날짜 (yyyy.mm.dd -> yyyy-mm-dd 형식으로 변경)
    original_date = safe_find_text(review_element, By.CSS_SELECTOR, ".sdp-review__article__list__info__product-info__reg-date")
    review_data['날짜'] = original_date.replace('.', '-') if original_date != "-" else original_date

    # 판매자 이름
    review_data['판매자_이름'] = safe_find_text(review_element, By.CSS_SELECTOR, ".sdp-review__article__list__info__product-info__seller_name")

    # 제품명
    review_data['제품명'] = safe_find_text(review_element, By.CSS_SELECTOR, ".sdp-review__article__list__info__product-info__name")

    # 리뷰 제목
    review_data['리뷰제목'] = safe_find_text(review_element, By.CSS_SELECTOR, ".sdp-review__article__list__headline")

    # 리뷰 내용
    review_data['리뷰내용'] = safe_find_text(review_element, By.CSS_SELECTOR, ".sdp-review__article__list__review__content")

    # 도움이 된 수
    review_data['도움됨_수'] = safe_find_text(review_element, By.CSS_SELECTOR, ".js_reviewArticleHelpfulCount")

    logger.info("이 리뷰의 스크래핑을 완료했습니다.")
    return review_data

def scrape_reviews(url, reviews_to_crawl, filename_base, sort_option):
    # URL 유효성 체크
    if "www.coupang.com" not in url:
        logger.error("잘못된 URL입니다. www.coupang.com이 포함된 URL을 입력하세요.")
        return []

    driver = setup_driver()
    driver.get(url)
    reviews = []
    current_page = 1
    reviews_per_page = 5  # 쿠팡의 경우 한 페이지당 5개의 리뷰가 있습니다
    pages_to_crawl = -(-reviews_to_crawl // reviews_per_page)  # 올림 나눗셈

    try:
        # 페이지가 완전히 로드될 때까지 대기
        WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        
        # 리뷰 탭 클릭
        review_tab_script = """
        const reviewTab = document.querySelector('#pdpReviewContentTab, #btfTab > ul.tab-titles > li:nth-child(2)');
        if (reviewTab) {
            reviewTab.click();
            return true;
        }
        return false;
        """
        is_clicked = driver.execute_script(review_tab_script)
        
        if not is_clicked:
            logger.error("리뷰 탭을 찾을 수 없습니다.")
            return reviews
        
        # 정렬 옵션 선택
        if sort_option == 2:  # 최신순
            logger.info("최신순 버튼을 클릭하고 있습니다...")
            try:
                newest_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, ".sdp-review__article__order__sort__newest-btn.js_reviewArticleNewListBtn.js_reviewArticleSortBtn"))
                )
                newest_button.click()
                time.sleep(2)
            except TimeoutException:
                logger.warning("최신순 버튼을 찾을 수 없습니다. 기본 정렬 순서로 진행합니다.")
        
        while len(reviews) < reviews_to_crawl and current_page <= pages_to_crawl:
            WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".sdp-review__article__list"))
            )
            
            # 스크롤 다운
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)

            review_elements = driver.find_elements(By.CSS_SELECTOR, ".sdp-review__article__list")
            logger.info(f"페이지 {current_page}: {len(review_elements)}개의 리뷰를 찾았습니다")

            # 순번 추가
            page_reviews = [scrape_review(driver, review_element, i + len(reviews)) for i, review_element in enumerate(review_elements)]
            reviews.extend(page_reviews[:reviews_to_crawl - len(reviews)])  # 필요한 만큼만 추가

            if len(reviews) % 100 == 0:
                save_to_csv(reviews, f"{filename_base}_중간저장_{len(reviews)}개.csv")
                logger.info(f"{len(reviews)}개의 리뷰가 중간 저장되었습니다.")

            if len(reviews) < reviews_to_crawl and current_page < pages_to_crawl:
                try:
                    if current_page % 10 == 0:
                        next_button = WebDriverWait(driver, 10).until(
                            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.sdp-review__article__page__next'))
                        )
                    else:
                        next_button = WebDriverWait(driver, 10).until(
                            EC.element_to_be_clickable((By.CSS_SELECTOR, f'button.sdp-review__article__page__num[data-page="{current_page + 1}"]'))
                        )
                    driver.execute_script("arguments[0].click();", next_button)
                    current_page += 1
                    time.sleep(3)
                except (NoSuchElementException, TimeoutException) as e:
                    logger.info(f"더 이상 페이지가 없습니다. 페이지 {current_page}에서 중지합니다. 오류: {str(e)}")
                    break
            else:
                logger.info(f"지정된 리뷰 수({reviews_to_crawl})에 도달했습니다")
                break

    except TimeoutException as e:
        logger.error(f"페이지 로딩 시간이 초과되었습니다: {str(e)}")
    except JavascriptException as e:
        logger.error(f"JavaScript 실행 중 오류가 발생했습니다: {str(e)}")
    except Exception as e:
        logger.error(f"스크래핑 중 오류가 발생했습니다: {str(e)}")
    finally:
        driver.quit()

    return reviews[:reviews_to_crawl]

def save_to_csv(reviews, filename):
    df = pd.DataFrame(reviews)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    logger.info(f"리뷰가 {filename} 파일로 저장되었습니다.")

def open_file(filename):
    if sys.platform.startswith('darwin'):  # macOS
        subprocess.call(('open', filename))
    elif sys.platform.startswith('win'):  # Windows
        os.startfile(filename)
    else:  # linux
        subprocess.call(('xdg-open', filename))

if __name__ == "__main__":
    url = input("크롤링할 상품 URL을 입력하세요: ")
    if "www.coupang.com" not in url:
        print("잘못된 URL입니다. www.coupang.com이 포함된 URL을 입력하세요.")
        sys.exit(1)
    
    reviews_to_crawl = int(input("크롤링할 리뷰 수를 입력하세요: "))
    filename_base = input("저장할 파일 이름을 입력하세요 (확장자 제외): ")
    sort_option = int(input("정렬 옵션을 선택하세요 (1: 베스트순, 2: 최신순): "))
    
    today_date = datetime.now().strftime("%Y%m%d")
    filename = f"{filename_base}_쿠팡후기_{'베스트순' if sort_option == 1 else '최신순'}_{today_date}.csv"
    
    scraped_reviews = scrape_reviews(url, reviews_to_crawl, filename_base, sort_option)

    if scraped_reviews:
        save_to_csv(scraped_reviews, filename)
        open_file(filename)
    else:
        print("리뷰를 스크래핑하지 못했습니다.")


크롤링할 상품 URL을 입력하세요: https://www.coupang.com/vp/products/6143634009?itemId=11805527089&vendorItemId=84881317474&q=%EB%9C%A8%EA%B0%9C%EC%8B%A4&itemsCount=36&searchId=b7b1a152e17c41dfa7b1b3a36034792e&rank=3&isAddedCart=
크롤링할 리뷰 수를 입력하세요: 738
저장할 파일 이름을 입력하세요 (확장자 제외): 우아한클릭 초보 뜨개질 코바늘 DIY 10종 세트
정렬 옵션을 선택하세요 (1: 베스트순, 2: 최신순): 2


2024-09-19 18:36:41,570 - INFO - 최신순 버튼을 클릭하고 있습니다...
2024-09-19 18:36:46,371 - INFO - 페이지 1: 5개의 리뷰를 찾았습니다
2024-09-19 18:36:46,371 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:36:46,462 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:36:46,462 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:36:46,525 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:36:46,525 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:36:46,577 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:36:46,577 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:36:46,637 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:36:46,637 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:36:46,697 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:36:51,736 - INFO - 페이지 2: 5개의 리뷰를 찾았습니다
2024-09-19 18:36:51,737 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:36:51,796 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:36:51,796 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:36:51,853 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:36:51,853 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:36:51,914 - INFO - 이 리뷰의 스크래핑을 완료했습니다

2024-09-19 18:38:01,298 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:38:01,369 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:38:01,369 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:38:01,440 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:38:01,440 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:38:01,497 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:38:01,497 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:38:01,553 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:38:01,554 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:38:01,610 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:38:06,670 - INFO - 페이지 16: 5개의 리뷰를 찾았습니다
2024-09-19 18:38:06,670 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:38:06,728 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:38:06,729 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:38:06,781 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:38:06,781 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:38:06,830 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:38:06,830 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:38:06,883 - INFO - 이 리뷰의 스크래핑을 완료했습니다.


2024-09-19 18:39:16,026 - INFO - 페이지 29: 5개의 리뷰를 찾았습니다
2024-09-19 18:39:16,027 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:39:16,082 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:39:16,083 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:39:16,144 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:39:16,144 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:39:16,198 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:39:16,198 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:39:16,252 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:39:16,252 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:39:16,313 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:39:21,355 - INFO - 페이지 30: 5개의 리뷰를 찾았습니다
2024-09-19 18:39:21,355 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:39:21,410 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:39:21,410 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:39:21,460 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:39:21,460 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:39:21,510 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:39:21,511 - INFO - 리뷰를 스크래핑하고 있습니다..

2024-09-19 18:40:25,659 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:40:30,701 - INFO - 페이지 43: 5개의 리뷰를 찾았습니다
2024-09-19 18:40:30,701 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:40:30,763 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:40:30,763 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:40:30,822 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:40:30,822 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:40:30,892 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:40:30,892 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:40:30,945 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:40:30,945 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:40:30,995 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:40:36,028 - INFO - 페이지 44: 5개의 리뷰를 찾았습니다
2024-09-19 18:40:36,029 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:40:36,094 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:40:36,094 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:40:36,161 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:40:36,162 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:40:36,212 - INFO - 이 리뷰의 스크래핑을 완료했습니

...

2024-09-19 18:44:09,847 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:44:14,900 - INFO - 페이지 85: 5개의 리뷰를 찾았습니다
2024-09-19 18:44:14,900 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:44:14,955 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:44:14,955 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:44:15,009 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:44:15,009 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:44:15,067 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:44:15,068 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:44:15,142 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:44:15,142 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:44:15,196 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:44:20,243 - INFO - 페이지 86: 5개의 리뷰를 찾았습니다
2024-09-19 18:44:20,243 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:44:20,296 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:44:20,297 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:44:20,351 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:44:20,351 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:44:20,413 - INFO - 이 리뷰의 스크래핑을 완료했습니

2024-09-19 18:45:29,778 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:45:29,832 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:45:29,832 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:45:29,885 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:45:29,885 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:45:29,949 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:45:29,950 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:45:30,004 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:45:30,005 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:45:30,061 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:45:35,099 - INFO - 페이지 100: 5개의 리뷰를 찾았습니다
2024-09-19 18:45:35,099 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:45:35,154 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:45:35,154 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:45:35,208 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:45:35,208 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:45:35,282 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:45:35,283 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:45:35,337 - INFO - 이 리뷰의 스크래핑을 완료했습니다.

2024-09-19 18:46:44,476 - INFO - 페이지 113: 5개의 리뷰를 찾았습니다
2024-09-19 18:46:44,476 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:46:44,531 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:46:44,531 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:46:44,580 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:46:44,580 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:46:44,638 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:46:44,639 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:46:44,687 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:46:44,688 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:46:44,737 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:46:49,779 - INFO - 페이지 114: 5개의 리뷰를 찾았습니다
2024-09-19 18:46:49,780 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:46:49,837 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:46:49,837 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:46:49,890 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:46:49,890 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:46:49,945 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:46:49,945 - INFO - 리뷰를 스크래핑하고 있습니다

2024-09-19 18:47:53,992 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:47:59,035 - INFO - 페이지 127: 5개의 리뷰를 찾았습니다
2024-09-19 18:47:59,035 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:47:59,096 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:47:59,096 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:47:59,146 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:47:59,146 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:47:59,214 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:47:59,214 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:47:59,263 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:47:59,264 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:47:59,326 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:48:04,363 - INFO - 페이지 128: 5개의 리뷰를 찾았습니다
2024-09-19 18:48:04,363 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:48:04,423 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:48:04,424 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:48:04,476 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:48:04,476 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:48:04,529 - INFO - 이 리뷰의 스크래핑을 완료했

2024-09-19 18:49:08,665 - INFO - 700개의 리뷰가 중간 저장되었습니다.
2024-09-19 18:49:13,711 - INFO - 페이지 141: 5개의 리뷰를 찾았습니다
2024-09-19 18:49:13,711 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:49:13,765 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:49:13,765 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:49:13,815 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:49:13,815 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:49:13,886 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:49:13,886 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:49:13,940 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:49:13,940 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:49:13,991 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:49:19,031 - INFO - 페이지 142: 5개의 리뷰를 찾았습니다
2024-09-19 18:49:19,032 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:49:19,089 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:49:19,089 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:49:19,138 - INFO - 이 리뷰의 스크래핑을 완료했습니다.
2024-09-19 18:49:19,138 - INFO - 리뷰를 스크래핑하고 있습니다...
2024-09-19 18:49:19,186 - INFO - 이 리뷰의 스크래핑을 완